## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [ ]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --no-cache-dir -q 2>/dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 146.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 281.9 MB/s eta 0:00:00


In [ ]:
!pip install -q tiktoken==0.6.0 pypdf==4.0.1 langchain==0.1.1 langchain-community==0.0.13 chromadb==0.4.22 sentence-transformers==2.3.1 huggingface_hub==0.23.2 numpy==1.25.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB

In [ ]:
!pip list | grep -E "llama|huggingface-hub|pandas|tiktoken|pymupdf|langchain|chromadb|sentence-transformers|numpy"

chromadb                                 0.4.22
geopandas                                1.1.1
huggingface-hub                          0.23.2
langchain                                0.1.1
langchain-community                      0.0.13
langchain-core                           0.1.23
langchain-text-splitters                 0.3.9
llama_cpp_python                         0.2.28
numpy                                    1.25.2
pandas                                   2.2.2
pandas-datareader                        0.10.0
pandas-gbq                               0.29.2
pandas-stubs                             2.2.2.240909
sentence-transformers                    2.3.1
sklearn-pandas                           2.2.0
tiktoken                                 0.6.0


In [ ]:
#Libraries for processing dataframes,text
import json
import os
import tiktoken
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader,PyPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Question Answering using LLM

#### Downloading and Loading the model

In [ ]:
# Step 1: Download a quantized model
model_path = hf_hub_download(
    repo_id="TheBloke/Llama-2-13B-chat-GGUF",
    filename="llama-2-13b-chat.Q5_K_M.gguf"  # pick quantization level you want
)

llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [ ]:
model_path

'/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf'

In [ ]:
# Step 2: Load the model
lcpp_llm = Llama(
    model_path=model_path,
    n_ctx=4096,  # Context window
    n_batch=1024,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43,  # Change this value based on your model and your GPU VRAM pool.
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


#### Response

In [ ]:
def response(query,max_tokens=1024,temperature=0,top_p=0.95,top_k=50):
    model_output = lcpp_llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
print(response("Q: What is the protocol for managing sepsis in a critical care unit?\nA:"))

Llama.generate: prefix-match hit


 The Surviving Sepsis Campaign (SSC) guidelines provide a standardized approach to managing sepsis in critical care units. Here are the key components of the protocol:

1. Early recognition and diagnosis:
	* Use a systematic approach to identify patients at risk for sepsis, such as the Sepsis Screening Tool (SST).
	* Evaluate patients with signs of infection, organ dysfunction, or clinical instability.
	* Use blood cultures and other diagnostic tests to confirm the presence of sepsis-causing pathogens.
2. Rapid administration of antibiotics:
	* Start broad-spectrum antibiotics within 1 hour of recognition of sepsis, ideally within 30 minutes.
	* Use a goal-directed approach to select antibiotics based on the suspected source of infection and the patient's allergies and sensitivities.
	* Monitor the patient's response to antibiotics and adjust the treatment as needed.
3. Supportive care:
	* Provide mechanical ventilation as needed to maintain adequate oxygenation.
	* Manage fluid status

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
print(response("Q: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?\nA:"))

Llama.generate: prefix-match hit


 Appendicitis is a medical emergency that requires prompt treatment. The common symptoms of appendicitis include:

1. Severe pain in the abdomen, usually starting near the belly button and then moving to the lower right side of the abdomen.
2. Nausea and vomiting.
3. Loss of appetite.
4. Fever.
5. Abdominal tenderness and guarding (muscle tension).
6. Abdominal swelling.
7. Diarrhea or constipation.

If you suspect that you or someone else may have appendicitis, it is essential to seek medical attention immediately. Appendicitis cannot be cured via medicine; instead, surgery is required to remove the inflamed appendix. The surgical procedure most commonly used to treat appendicitis is an appendectomy, which involves removing the inflamed appendix through a small incision in the abdomen.

There are two types of appendectomies: open and laparoscopic. Open appendectomy is the traditional method, where a larger incision is made to allow for better visualization of the appendix. Laparoscopi

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
print(response("Q: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?\nA:"))

Llama.generate: prefix-match hit


 Sudden patchy hair loss, also known as alopecia areata, can be challenging to treat, but there are several effective solutions and treatments that can help promote hair growth and reduce inflammation. Here are some of the most common treatment options for sudden patchy hair loss:

1. Topical corticosteroids: Over-the-counter or prescription topical corticosteroids can help reduce inflammation and promote hair growth. These medications are applied directly to the affected area, and they can be effective in mild to moderate cases of alopecia areata.
2. Minoxidil: Minoxidil (Rogaine) is a solution that is applied to the scalp to stimulate hair growth. It is available over-the-counter and has been shown to be effective in treating patchy hair loss.
3. Anthralin: Anthralin is a medication that is applied to the skin to reduce inflammation and promote hair growth. It is often used in combination with corticosteroids.
4. Phototherapy: Exposure to specific wavelengths of light, such as ultrav

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
print(response("Q: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?\nA:"))

Llama.generate: prefix-match hit


 Treatments for a person who has sustained a physical injury to brain tissue and experienced temporary or permanent impairment of brain function may include:

1. Medications: To manage symptoms such as pain, anxiety, and depression.
2. Rehabilitation therapy: To improve cognitive, motor, and behavioral functions.
3. Surgery: To repair damaged tissue or relieve pressure on the brain.
4. Lifestyle modifications: To accommodate any long-term changes in brain function, such as adjusting work or living situations.
5. Cognitive training: To improve memory, attention, and other cognitive functions.
6. Behavioral therapy: To address any emotional or behavioral changes resulting from the injury.
7. Neuropsychological rehabilitation: To help the individual adapt to any permanent changes in brain function and improve their quality of life.
8. Stem cell therapy: To promote healing and repair of damaged brain tissue.
9. Hyperbaric oxygen therapy: To increase oxygen flow to the brain and promote hea

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
print(response("Q: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?\nA:"))

Llama.generate: prefix-match hit


 If someone has fractured their leg while hiking, it is essential to take immediate action to provide proper care and ensure their recovery. Here are some necessary precautions and treatment steps:

1. Stop activity: Immediately stop any strenuous activity to prevent further injury or damage to the affected area.
2. Assess the injury: Evaluate the severity of the fracture by checking for signs of swelling, deformity, or open wounds. If the fracture is severe or if there are any open wounds, call for emergency medical assistance.
3. Immobilize the affected area: Use a splint or brace to immobilize the leg and prevent further movement. This can help reduce pain and prevent further damage.
4. Apply ice: Apply ice packs to the affected area to reduce swelling and pain. Wrap the ice in a towel or cloth to avoid direct contact with the skin.
5. Elevate the affected area: Elevate the leg above the level of the heart to reduce swelling and improve blood flow.
6. Monitor vital signs: Check the 

### Observations

**Quality & Structure**
- Well-structured with clear numbering and bullet points  
- Clinically relevant order maintained  
- Good depth while staying focused  

**Medical Accuracy**
- Aligns with standard protocols & guidelines  
- Correct use of medical terminology  
- Treatments progress from conservative to invasive  

**Comprehensiveness**
- **Sepsis:** Complete SSC emergency care pathway  
- **Appendicitis:** Clear symptoms vs. surgical intervention  
- **Hair Loss:** Balanced causes and treatments  
- **Brain Injury:** Multi-modal treatment & rehab  
- **Leg Fracture:** Immediate care and long-term management  

**Areas for Enhancement**
- Add more specifics on treatment timeframes  
- Clarify emergency contact guidelines  
- Consider cost implications  
- Prioritize treatments more clearly  

**Notable Strengths**
- Consistent emphasis on professional medical help  
- Balanced technical vs. accessible language  
- Covers immediate and long-term care  
- Holistic view: physical & psychological aspects  

## Question Answering using LLM with Prompt Engineering

In [ ]:
questions = [
    "What is the protocol for managing sepsis in a critical care unit?",
    "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
]

prompt_variants = [
    lambda q: f"""You are a highly reliable and cautious medical assistant.
You must only provide evidence-based answers derived strictly from The Merck Manual (Nineteenth Edition).
If the Merck Manual does not clearly cover the topic,
You must only provide evidence-based answers following standard medical guidelines (WHO, CDC, NICE).
If you are not sure, say "This requires specialist consultation." rather than guessing.
Do not guess or provide information from non authentic medical sources.
Always include reasoning and a clear, structured answer.

Question: {q}
Answer:
""", #Safety + Evidence-Based + Fallback

    lambda q: f"""Provide your answer based only on information from the The Merck Manual (Nineteenth Edition),
    if you don't find that data then only provide information you are confident is supported by medical guidelines
and use the following format:

1. Symptoms/Presentation
2. Diagnosis Steps
3. Treatment Protocol
4. Cautions / Follow-up

If you do not get clear information, respond with:
"This requires specialist consultation."

Question: {q}
Answer:
""" #Structured Answer Format
]

param_combos = [
    {"temperature": 0.2, "top_p": 0.9, "top_k": 40, "max_tokens": 1024},  #Very factual, less diverse. Useful for guidelines
    {"temperature": 0.5, "top_p": 0.8, "top_k": 20, "max_tokens": 1024},  # Balanced reasoning
    {"temperature": 0.7, "top_p": 0.95, "top_k": 50, "max_tokens": 1024}, # Fluent + accurate, flexible
]


results = []

for qi, q in enumerate(questions):
    for pi, prompt_func in enumerate(prompt_variants):
        prompt = prompt_func(q)
        for ci, params in enumerate(param_combos):
            answer = response(query=prompt,
                              max_tokens=params["max_tokens"],
                              temperature=params["temperature"],
                              top_p=params["top_p"],
                              top_k=params["top_k"])
            results.append({
                "question": q,
                "prompt_variant": pi,
                "params": params,
                "answer": answer
            })

import pandas as pd
df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


In [ ]:
file_path = '/content/drive/MyDrive/promptengg_results.pkl'
df.to_pickle(file_path)

In [ ]:
df.head(6)

,question,prompt_variant,params,answer
0,What is the protocol for managing sepsis in a critical care unit?,0,"{'temperature': 0.2, 'top_p': 0.9, 'top_k': 40, 'max_tokens': 1024}","\nAccording to The Merck Manual (Nineteenth Edition), the management of sepsis in a critical care unit involves a multidisciplinary approach that includes early recognition, prompt administration of antibiotics and supportive care, and close monitoring of vital signs and laboratory results. Here is a step-by-step protocol for managing sepsis in a critical care unit:\n\nStep 1: Early Recognition (within 1 hour)\n\n* Assess the patient's vital signs, including temperature, blood pressure, tachycardia, and tachypnea.\n* Check for signs of infection, such as fever, chills, or purulent drainage.\n* Evaluate the patient's level of consciousness and mental status.\n\nStep 2: Administration of Antibiotics (within 1 hour)\n\n* Administer broad-spectrum antibiotics effective against common causes of sepsis, such as Gram-positive and Gram-negative bacteria, and fungi.\n* Use a loading dose of antibiotics followed by maintenance doses every 8-12 hours.\n* Monitor the patient's response to antibiotics and adjust the dosage or switch medications as needed.\n\nStep 3: Supportive Care (within 1 hour)\n\n* Provide oxygen therapy to maintain oxygen saturation levels above 90%.\n* Administer fluids and electrolytes to maintain hydration and prevent hypovolemia.\n* Monitor the patient's blood glucose levels and administer insulin or dextrose as needed.\n* Use a vasopressor to maintain mean arterial pressure (MAP) above 65 mmHg if the patient is hypotensive.\n\nStep 4: Close Monitoring (every 2-4 hours)\n\n* Assess the patient's vital signs, including temperature, blood pressure, tachycardia, and tachypnea.\n* Check for signs of organ dysfunction, such as altered mental status, respiratory distress, or renal failure.\n* Monitor the patient's laboratory results, including white blood cell count, serum creatinine, and blood urea nitrogen.\n\nStep 5: Adjust Therapy (as needed)\n\n* Adjust antibiotic dosages or switch medications based on the patient's response to treatment.\n* Provide additional supportive care as needed, such as mechanical ventilation or dialysis.\n* Consult a specialist if the patient's condition deteriorates or if there are signs of organ failure.\n\nThis protocol is based on the latest evidence-based guidelines from reputable medical organizations such as the World Health Organization (WHO), the Centers for Disease Control and Prevention (CDC), and the National Institute for Health and Care Excellence (NICE). It is important to note that this protocol should be tailored to the individual patient's needs and may require modification based on the patient's specific condition and medical history."
1,What is the protocol for managing sepsis in a critical care unit?,0,"{'temperature': 0.5, 'top_p': 0.8, 'top_k': 20, 'max_tokens': 1024}","\nAccording to The Merck Manual (Nineteenth Edition), the protocol for managing sepsis in a critical care unit involves a multidisciplinary approach that includes early recognition, prompt administration of antibiotics and fluid resuscitation, and close monitoring of vital signs and laboratory values.\n\nThe Sepsis-3 definition should be used to diagnose sepsis (1). The clinical criteria for sepsis include suspected or confirmed infection, organ dysfunction, and a high serum lactate level (2).\n\nThe first step in managing sepsis is to identify the source of infection and provide appropriate antibiotics. Broad-spectrum antibiotics should be administered as soon as possible, ideally within the first hour of recognition of sepsis (3). The choice of antibiotics should be guided by the suspected pathogen and the patient's allergies and medical history.\n\nIn addition to antibiotics, fluid resuscitation is critical in managing sepsis. Crystalloid fluids such as normal saline or lactated Ringer's solution should be administered in

In [ ]:
#answers_df = pd.read_pickle(file_path)

In [ ]:
answers_df = df.copy()

### Query 1: What is the protocol for managing sepsis in a critical care unit?

#### Evidence based prompt

**Parameter Combination**
 - temperature: 0.2, top_p: 0.9, top_k: 40, max_tokens: 1024 **(Very factual, less diverse. Useful for guidelines)**


In [ ]:
row = answers_df[(answers_df['question'] == 'What is the protocol for managing sepsis in a critical care unit?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[0])]
print(row['answer'].values[0])


According to The Merck Manual (Nineteenth Edition), the management of sepsis in a critical care unit involves a multidisciplinary approach that includes early recognition, prompt administration of antibiotics and supportive care, and close monitoring of vital signs and laboratory results. Here is a step-by-step protocol for managing sepsis in a critical care unit:

Step 1: Early Recognition (within 1 hour)

* Assess the patient's vital signs, including temperature, blood pressure, tachycardia, and tachypnea.
* Check for signs of infection, such as fever, chills, or purulent drainage.
* Evaluate the patient's level of consciousness and mental status.

Step 2: Administration of Antibiotics (within 1 hour)

* Administer broad-spectrum antibiotics effective against common causes of sepsis, such as Gram-positive and Gram-negative bacteria, and fungi.
* Use a loading dose of antibiotics followed by maintenance doses every 8-12 hours.
* Monitor the patient's response to antibiotics and adjus

**Parameter Combination**
 - temperature: 0.5, top_p: 0.8, top_k: 20, max_tokens: 1024 (**Balanced reasoning**)

In [ ]:
row = answers_df[(answers_df['question'] == 'What is the protocol for managing sepsis in a critical care unit?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[1])]
print(row['answer'].values[0])


According to The Merck Manual (Nineteenth Edition), the protocol for managing sepsis in a critical care unit involves a multidisciplinary approach that includes early recognition, prompt administration of antibiotics and fluid resuscitation, and close monitoring of vital signs and laboratory values.

The Sepsis-3 definition should be used to diagnose sepsis (1). The clinical criteria for sepsis include suspected or confirmed infection, organ dysfunction, and a high serum lactate level (2).

The first step in managing sepsis is to identify the source of infection and provide appropriate antibiotics. Broad-spectrum antibiotics should be administered as soon as possible, ideally within the first hour of recognition of sepsis (3). The choice of antibiotics should be guided by the suspected pathogen and the patient's allergies and medical history.

In addition to antibiotics, fluid resuscitation is critical in managing sepsis. Crystalloid fluids such as normal saline or lactated Ringer's s

**Parameter Combination**
 - temperature: 0.7, top_p: 0.95, top_k: 50, max_tokens: 1024 (**Fluent, accurate, flexible**)

In [ ]:
row = answers_df[(answers_df['question'] == 'What is the protocol for managing sepsis in a critical care unit?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[2])]
print(row['answer'].values[0])


This requires specialist consultation. The management of sepsis in a critical care unit should be guided by established clinical practice guidelines such as those from the Surviving Sepsis Campaign (SSC) and the Infectious Diseases Society of America (IDSA). These guidelines provide evidence-based recommendations for early recognition, diagnosis, and treatment of sepsis.

In addition to these guidelines, the following protocol should be followed in a critical care unit:

1. Early recognition and prompt administration of antibiotics: The SSC recommends that antibiotics be administered within 1 hour of recognition of sepsis, and that broad-spectrum antibiotics be used as the initial treatment for severe sepsis and septic shock.
2. Aggressive fluid resuscitation: Patients with sepsis should receive aggressive fluid resuscitation to restore intravascular volume and maintain mean arterial pressure (MAP) ≥ 65 mmHg.
3. Vasopressor administration: Vasopressors should be used to maintain MAP ≥

#### Structured prompt

**Parameter Combination**
 - temperature: 0.2, top_p: 0.9, top_k: 40, max_tokens: 1024 **(Very factual, less diverse. Useful for guidelines)**

In [ ]:
row = answers_df[(answers_df['question'] == 'What is the protocol for managing sepsis in a critical care unit?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[0])]
print(row['answer'].values[0])


1. Symptoms/Presentation: Sepsis is a life-threatening organ dysfunction caused by the host's systemic inflammatory response to a pathogen, with signs of organ dysfunction and risk of death. The presentation may vary depending on the underlying infection, but common symptoms include fever, tachycardia, tachypnea, confusion, and hypotension.
2. Diagnosis Steps: The diagnosis of sepsis is based on clinical criteria, including systemic inflammatory response syndrome (SIRS) criteria, and laboratory confirmation of an infection. SIRS criteria include body temperature >38°C or <36°C, heart rate >90 beats per minute, respiratory rate >20 breaths per minute, or white blood cell count >12,000 cells/mm3 or <4,000 cells/mm3. Laboratory confirmation of an infection may include blood cultures, urinalysis, or other tests to identify the causative pathogen.
3. Treatment Protocol: The treatment of sepsis is based on early recognition and aggressive management of the underlying infection, fluid resusc

**Parameter Combination**
 - temperature: 0.5, top_p: 0.8, top_k: 20, max_tokens: 1024 (**Balanced reasoning**)

In [ ]:
row = answers_df[(answers_df['question'] == 'What is the protocol for managing sepsis in a critical care unit?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[1])]
print(row['answer'].values[0])

1. Symptoms/Presentation: Sepsis is characterized by systemic inflammation and organ dysfunction, which can lead to signs such as fever, tachycardia, tachypnea, confusion, and hypotension. The presence of sepsis is often indicated by a severe infection, such as pneumonia, urinary tract infection, or bloodstream infection.
2. Diagnosis Steps: To diagnose sepsis, the following steps should be taken:
* Assess the patient's vital signs, including temperature, blood pressure, tachycardia, and tachypnea.
* Check for signs of organ dysfunction, such as altered mental status, respiratory failure, cardiovascular instability, and renal insufficiency.
* Obtain blood cultures to identify the source of infection.
* Measure serum lactate levels to assess tissue perfusion.
* Perform a complete blood count (CBC) to evaluate for leukocytosis or leukopenia.
3. Treatment Protocol: The treatment of sepsis should be guided by the Surviving Sepsis Campaign (SSC) guidelines, which recommend the following ste

**Parameter Combination**
 - temperature: 0.7, top_p: 0.95, top_k: 50, max_tokens: 1024 (**Fluent, accurate, flexible**)

In [ ]:
row = answers_df[(answers_df['question'] == 'What is the protocol for managing sepsis in a critical care unit?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[2])]
print(row['answer'].values[0])


1. Symptoms/Presentation: Sepsis is a systemic inflammatory response to an infection, characterized by fever, tachycardia, tachypnea, and signs of organ dysfunction (e.g., altered mentation, confusion, or renal failure).
2. Diagnosis Steps: Confirm the presence of an infection and identify its source. Look for signs of organ dysfunction, such as abnormal lab values (e.g., WBC count, creatinine, blood urea nitrogen) and changes in vital signs (e.g., tachycardia, tachypnea).
3. Treatment Protocol: Administer broad-spectrum antibiotics and fluid resuscitation promptly. Use vasopressors to maintain mean arterial pressure ≥65 mmHg if systolic blood pressure is <90 mmHg despite fluid resuscitation. Consider corticosteroids for severe sepsis or septic shock. Monitor for signs of organ dysfunction and adjust therapy accordingly.
4. Cautions/Follow-up: Closely monitor the patient's condition, including vital signs, laboratory values, and signs of organ dysfunction. Consider escalating therapy 

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

#### Evidence based prompt

**Parameter Combination**
 - temperature: 0.2, top_p: 0.9, top_k: 40, max_tokens: 1024 **(Very factual, less diverse. Useful for guidelines)**


In [ ]:
row = answers_df[(answers_df['question'] == 'What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[0])]
print(row['answer'].values[0])



Symptoms of Appendicitis:

The most common symptoms of appendicitis include:

1. Sudden and severe pain in the abdomen, usually starting near the belly button and then moving to the lower right side of the abdomen.
2. Nausea and vomiting.
3. Loss of appetite.
4. Fever.
5. Abdominal tenderness and guarding (muscle tension).
6. Abdominal swelling.
7. Diarrhea or constipation.

Note: These symptoms can be similar to other conditions such as a ruptured ovarian cyst, diverticulitis, or a blockage in the intestine. Therefore, it is essential to seek medical attention immediately if you experience these symptoms.

Treatment of Appendicitis:

Appendicitis cannot be cured via medicine; surgical intervention is necessary to treat it. The standard surgical procedure for appendicitis is an appendectomy, which involves the removal of the inflamed appendix. There are two types of appendectomies:

1. Open Appendectomy: This is the traditional method, where a single incision is made in the lower righ

**Parameter Combination**
 - temperature: 0.5, top_p: 0.8, top_k: 20, max_tokens: 1024 (**Balanced reasoning**)

In [ ]:
row = answers_df[(answers_df['question'] == 'What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[1])]
print(row['answer'].values[0])


According to The Merck Manual (Nineteenth Edition), the common symptoms of appendicitis include:

* Sudden onset of severe pain in the lower right abdomen that begins around the navel and then moves to the lower right abdomen.
* Nausea and vomiting
* Loss of appetite
* Fever
* Abdominal tenderness and guarding (muscle tension)
* Abdominal swelling

It is important to note that not everyone with appendicitis will experience all of these symptoms, and some people may have different symptoms altogether.

Regarding treatment, The Merck Manual states that appendicitis cannot be cured via medicine. Instead, surgical intervention is necessary to remove the inflamed appendix. The recommended surgical procedure for appendicitis is an appendectomy, which can be performed through a laparoscopic (minimally invasive) or open incision.

In laparoscopic appendectomy, several small incisions are made in the abdomen, and a laparoscope (a thin tube with a camera and light) is inserted to visualize the 

**Parameter Combination**
 - temperature: 0.7, top_p: 0.95, top_k: 50, max_tokens: 1024 (**Fluent, accurate, flexible**)

In [ ]:
row = answers_df[(answers_df['question'] == 'What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[2])]
print(row['answer'].values[0])


Symptoms of appendicitis can vary from patient to patient. However, The Merck Manual (Nineteenth Edition) lists the most common symptoms as follows:

* Sudden onset of severe pain in the lower right abdomen that begins near the navel and then moves to the lower right abdomen
* Nausea and vomiting
* Loss of appetite
* Fever
* Abdominal tenderness and guarding (muscle tension)
* Abdominal swelling

It is important to note that not everyone with appendicitis will experience all of these symptoms, and some people may have different symptoms altogether. However, if a patient presents with at least two of these symptoms, it is highly likely that they are experiencing appendicitis.

Regarding treatment, The Merck Manual (Nineteenth Edition) states that appendicitis cannot be cured via medicine. Instead, surgical intervention is necessary to treat the condition. The standard surgical procedure for treating appendicitis is an appendectomy, which involves removing the inflamed appendix.

There 

#### Structured prompt

**Parameter Combination**
 - temperature: 0.2, top_p: 0.9, top_k: 40, max_tokens: 1024 **(Very factual, less diverse. Useful for guidelines)**


In [ ]:
row = answers_df[(answers_df['question'] == 'What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[0])]
print(row['answer'].values[0])


1. Symptoms/Presentation:  
The most common symptoms of appendicitis include sudden onset of severe pain in the lower right abdomen, often with nausea, vomiting, fever, and loss of appetite. The pain may begin near the navel and then move to the lower right abdomen. Other symptoms may include:  
* Abdominal tenderness and guarding (muscle tension)
* Rigidity in the abdominal muscles
* Loss of bowel sounds
* Palpable mass in the right lower quadrant of the abdomen
2. Diagnosis Steps:  
The diagnosis of appendicitis is based on a combination of symptoms, physical examination findings, and imaging studies. The following steps are typically used to diagnose appendicitis:  
* Medical history and physical examination to identify the presence of symptoms and rule out other possible causes of abdominal pain.
* Laboratory tests, such as a complete blood count (CBC) and a urinalysis, to look for signs of infection and inflammation.
* Imaging studies, such as an X-ray or CT scan, to confirm the 

**Parameter Combination**
 - temperature: 0.5, top_p: 0.8, top_k: 20, max_tokens: 1024 (**Balanced reasoning**)


In [ ]:
row = answers_df[(answers_df['question'] == 'What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[1])]
print(row['answer'].values[0])


1. Symptoms/Presentation:  
* Severe pain in the lower right abdomen that begins suddenly and worsens over time
* Nausea and vomiting
* Loss of appetite
* Fever
* Abdominal tenderness and guarding (muscle tension)
* Rigidity (stiffness) in the abdomen
* Palpable (able to be felt) mass in the right lower quadrant of the abdomen
2. Diagnosis Steps:  
* Physical examination and medical history
* Blood tests (such as a complete blood count and electrolyte panel) to rule out other conditions
* Imaging studies (such as X-rays or CT scans) to confirm the diagnosis and assess the severity of the inflammation
3. Treatment Protocol:  
* Appendectomy (surgical removal of the appendix), which is the standard treatment for appendicitis
	+ Open appendectomy: A traditional surgical procedure in which a single incision is made in the lower right abdomen to remove the inflamed appendix
	+ Laparoscopic appendectomy: A minimally invasive surgical procedure in which several small incisions are made in th

**Parameter Combination**
 - temperature: 0.7, top_p: 0.95, top_k: 50, max_tokens: 1024 (**Fluent, accurate, flexible**)


In [ ]:
row = answers_df[(answers_df['question'] == 'What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[2])]
print(row['answer'].values[0])


1. Symptoms/Presentation: 
The most common symptoms of appendicitis include:  
	* Sudden onset of severe pain in the lower right abdomen  
	* Nausea and vomiting  
	* Loss of appetite  
	* Fever  
	* Abdominal tenderness and guarding (muscle tension)  
	* Palpable mass in the right lower quadrant  

It's important to note that appendicitis can be difficult to diagnose, especially in its early stages. The symptoms may be subtle or nonspecific, and they may resemble other conditions such as a simple stomachache or urinary tract infection.  

2. Diagnosis Steps: 
The diagnosis of appendicitis is based on a combination of physical examination findings, medical history, and imaging studies. The following steps are commonly used to diagnose appendicitis:  
	* Complete blood count (CBC) and blood culture to rule out infection  
	* Urinalysis to rule out urinary tract infection  
	* Imaging studies such as X-ray, ultrasound, or CT scan to confirm the presence of an inflamed appendix  
	* Lapa

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

#### Evidence based prompt

**Parameter Combination**
 - temperature: 0.2, top_p: 0.9, top_k: 40, max_tokens: 1024 **(Very factual, less diverse. Useful for guidelines)**


In [ ]:
row = answers_df[(answers_df['question'] == 'What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[0])]
print(row['answer'].values[0])


The Merck Manual (Nineteenth Edition) does not provide a specific section on sudden patchy hair loss. However, based on standard medical guidelines from WHO, CDC, and NICE, here are some possible causes and effective treatments for localized bald spots on the scalp:

Possible Causes:

1. Alopecia Areata - an autoimmune condition that causes hair loss in patches.
2. Telogen Effluvium - a stress-induced condition that causes excessive hair shedding, often resulting in localized bald spots.
3. Folliculitis - an inflammation of the hair follicles, which can lead to hair loss in specific areas.
4. Tinea Capitis (Ringworm) - a fungal infection that can cause patchy hair loss on the scalp.
5. Traction Alopecia - hair loss due to constant pulling or tension on the hair shaft, often seen in individuals with tight hairstyles.

Effective Treatments:

1. Corticosteroid Injections - can help stimulate hair growth and reduce inflammation in cases of alopecia areata and folliculitis.
2. Topical Mino

**Parameter Combination**
 - temperature: 0.5, top_p: 0.8, top_k: 20, max_tokens: 1024 (**Balanced reasoning**)


In [ ]:
row = answers_df[(answers_df['question'] == 'What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[1])]
print(row['answer'].values[0])


The Merck Manual (Nineteenth Edition) does not provide a specific section on sudden patchy hair loss. However, based on standard medical guidelines from WHO, CDC, and NICE, here are some possible causes and effective treatments for localized bald spots on the scalp:

Possible Causes:

1. Telogen effluvium: This is a common cause of sudden patchy hair loss, often triggered by physical or emotional stress. It results in an increase in telogen hair follicles, leading to excessive shedding of hair.
2. Alopecia areata: This is an autoimmune condition that can cause sudden hair loss in patches or over the entire scalp. It occurs when the immune system mistakenly attacks healthy hair follicles, leading to hair loss.
3. Folliculitis decalvans: This is a rare condition characterized by recurring bacterial infections of the hair follicles, leading to scarring alopecia and patchy hair loss.
4. Tinea capitis (ringworm): This is a fungal infection that can cause patchy hair loss, especially in chi

**Parameter Combination**
 - temperature: 0.7, top_p: 0.95, top_k: 50, max_tokens: 1024 (**Fluent, accurate, flexible**)


In [ ]:
row = answers_df[(answers_df['question'] == 'What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[2])]
print(row['answer'].values[0])


Based on The Merck Manual (Nineteenth Edition), there are several effective treatments and solutions for addressing sudden patchy hair loss or localized bald spots on the scalp. Here are some of the possible causes and their corresponding treatments:

1. Telogen effluvium: This is a common cause of sudden patchy hair loss, particularly in women. It occurs when there is an increase in the number of hair follicles that stop growing and enter the resting phase (telogen phase). Treatment includes addressing underlying stressors, reducing stress levels, and using minoxidil 2% solution or finasteride medication.
2. Alopecia areata: This is an autoimmune condition that can cause sudden patchy hair loss. Treatment options include topical corticosteroids, intralesional corticosteroids, and immunomodulatory therapy with diphencyprone (DPCP).
3. Folliculitis decalvans: This is a condition that causes scarring alopecia and hair loss in patches. Treatment includes antibiotics, topical corticostero

#### Structured prompt

**Parameter Combination**
 - temperature: 0.2, top_p: 0.9, top_k: 40, max_tokens: 1024 **(Very factual, less diverse. Useful for guidelines)**

In [ ]:
row = answers_df[(answers_df['question'] == 'What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[0])]
print(row['answer'].values[0])


1. Symptoms/Presentation: Sudden patchy hair loss, also known as alopecia areata, typically presents with well-defined, smooth, round or oval bald patches on the scalp. The affected areas may be devoid of any inflammation or scarring. Hair loss may be partial or total, and can occur suddenly or gradually over time.
2. Diagnosis Steps: To diagnose sudden patchy hair loss, a thorough medical history and physical examination are essential. A dermatologist may perform a scalp biopsy to rule out other conditions such as psoriasis or eczema. Blood tests may be ordered to check for thyroid disorders, autoimmune diseases, or anemia.
3. Treatment Protocol: The treatment approach for sudden patchy hair loss depends on the underlying cause and severity of the condition. In mild cases, watchful waiting with close monitoring may be recommended. Topical corticosteroids, minoxidil, and anthralin may be prescribed for localized application to promote hair growth and reduce inflammation. In severe cas

**Parameter Combination**
 - temperature: 0.5, top_p: 0.8, top_k: 20, max_tokens: 1024 (**Balanced reasoning**)


In [ ]:
row = answers_df[(answers_df['question'] == 'What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[1])]
print(row['answer'].values[0])


1. Symptoms/Presentation: Sudden patchy hair loss, also known as alopecia areata, typically presents as localized bald spots on the scalp, eyebrows, or beard. The hair loss is usually sudden and can range from small to large patches. The affected skin may be smooth and normal in appearance, with no scaling or redness.
2. Diagnosis Steps: To diagnose alopecia areata, a healthcare provider will typically perform a physical examination and ask questions about the patient's medical history. They may also use a scalp biopsy to confirm the diagnosis. The healthcare provider may also rule out other conditions that can cause similar symptoms, such as tinea capitis (ringworm of the scalp) or lupus erythematosus.
3. Treatment Protocol: The treatment for alopecia areata depends on the severity and location of the hair loss. For small patches, treatment may not be necessary, and the condition may resolve on its own. Topical corticosteroids, such as hydrocortisone cream or lotion, can be applied d

**Parameter Combination**
 - temperature: 0.7, top_p: 0.95, top_k: 50, max_tokens: 1024 (**Fluent, accurate, flexible**)


In [ ]:
row = answers_df[(answers_df['question'] == 'What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[2])]
print(row['answer'].values[0])


1. Symptoms/Presentation: Sudden patchy hair loss, also known as alopecia areata, typically presents with well-defined, round or oval patches of hair loss on the scalp. The hair loss may be localized to one area or may be more widespread. The affected skin may appear normal, with no scales, redness, or itching.
2. Diagnosis Steps: To diagnose sudden patchy hair loss, a thorough medical history and physical examination are necessary. A skin scraping or biopsy may be performed to confirm the diagnosis of alopecia areata. Other conditions that cause similar symptoms, such as tinea capitis (ringworm), eczema, psoriasis, or lupus erythematosus, should be ruled out.
3. Treatment Protocol: The treatment for sudden patchy hair loss depends on the severity and extent of the hair loss. In mild cases, no treatment may be necessary, and the condition may resolve spontaneously. Topical corticosteroids, such as hydrocortisone cream or lotion, may be prescribed to reduce inflammation and promote hai

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

#### Evidence based prompt

**Parameter Combination**
 - temperature: 0.2, top_p: 0.9, top_k: 40, max_tokens: 1024 **(Very factual, less diverse. Useful for guidelines)**

In [ ]:
row = answers_df[(answers_df['question'] == 'What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[0])]
print(row['answer'].values[0])


This requires specialist consultation. The treatment for a person with a physical injury to brain tissue depends on the severity and location of the injury, as well as the extent of any resulting impairment of brain function. The Merck Manual does not provide specific guidance on this topic, as it is outside the scope of primary care medicine. However, based on standard medical guidelines from the World Health Organization (WHO), the Centers for Disease Control and Prevention (CDC), and the National Institute for Health and Care Excellence (NICE), the following general principles may be applicable:

1. Emergency care: If the injury is recent and severe, emergency medical care should be sought immediately. This may include surgical intervention to relieve pressure on the brain or repair damaged blood vessels.
2. Rehabilitation: Depending on the severity of the injury, rehabilitation may be necessary to help regain lost function and improve quality of life. This may include physical the

**Parameter Combination**
 - temperature: 0.5, top_p: 0.8, top_k: 20, max_tokens: 1024 (**Balanced reasoning**)

In [ ]:
row = answers_df[(answers_df['question'] == 'What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[1])]
print(row['answer'].values[0])


According to The Merck Manual (Nineteenth Edition), the treatment for a person who has sustained a physical injury to brain tissue and is experiencing temporary or permanent impairment of brain function will depend on the severity and location of the injury. 
The following treatments may be recommended:

1. Immediate medical attention: The individual should receive immediate medical attention, including stabilization of vital signs and management of any life-threatening complications.  
2. Neuroimaging studies: Neuroimaging studies such as CT or MRI scans may be ordered to evaluate the extent and location of the injury.  
3. Supportive care: Supportive care, such as mechanical ventilation, fluids, and electrolyte replacement, may be necessary to maintain the individual's vital functions.  
4. Medications: Medications such as analgesics, sedatives, and anticonvulsants may be prescribed to manage symptoms such as pain, agitation, and seizures.  
5. Rehabilitation therapy: Rehabilitation

**Parameter Combination**
 - temperature: 0.7, top_p: 0.95, top_k: 50, max_tokens: 1024 (**Fluent, accurate, flexible**)

In [ ]:
row = answers_df[(answers_df['question'] == 'What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[2])]
print(row['answer'].values[0])


This requires specialist consultation.  

Please provide additional information regarding the type and severity of the injury, as well as any other relevant medical history.


#### Structured prompt

**Parameter Combination**
 - temperature: 0.2, top_p: 0.9, top_k: 40, max_tokens: 1024 **(Very factual, less diverse. Useful for guidelines)**


In [ ]:
row = answers_df[(answers_df['question'] == 'What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[0])]
print(row['answer'].values[0])


1. Symptoms/Presentation: The symptoms of a brain injury can vary depending on the location and severity of the injury. Common presenting symptoms may include headache, dizziness, confusion, memory loss, difficulty with speech and language, weakness or paralysis of specific limbs, and changes in behavior or mood.  
2. Diagnosis Steps: The diagnosis of a brain injury is typically made through a combination of physical examination, imaging studies such as CT or MRI scans, and laboratory tests to rule out other conditions that may cause similar symptoms.  
3. Treatment Protocol: The treatment of a brain injury depends on the severity and location of the injury, as well as the specific symptoms presented by the patient. In general, the goals of treatment are to stabilize the patient, manage any underlying causes of the injury, and promote healing and recovery. This may involve medications such as pain relievers, anti-seizure drugs, and steroids; surgical intervention to relieve pressure o

**Parameter Combination**
 - temperature: 0.5, top_p: 0.8, top_k: 20, max_tokens: 1024 (**Balanced reasoning**)


In [ ]:
row = answers_df[(answers_df['question'] == 'What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[1])]
print(row['answer'].values[0])


1. Symptoms/Presentation:  
Depending on the location and severity of the injury, symptoms may include headache, dizziness, confusion, weakness or paralysis of specific limbs, difficulty with speech and language, memory loss, and changes in behavior or personality.  

2. Diagnosis Steps:  
The diagnosis is based on a combination of physical examination, imaging studies such as CT or MRI scans, and laboratory tests to rule out other conditions that may cause similar symptoms.  

3. Treatment Protocol:  
Treatment for brain injury depends on the severity and location of the injury, as well as the age and overall health of the patient. In general, treatment may include: 
a) Medications to manage symptoms such as pain, seizures, and inflammation. 
b) Rehabilitation therapy to improve cognitive, physical, and behavioral functions. 
c) Surgery to relieve pressure or repair damaged tissue. 
d) Supportive care to address specific complications such as fluid and electrolyte imbalances, infecti

**Parameter Combination**
 - temperature: 0.7, top_p: 0.95, top_k: 50, max_tokens: 1024 (**Fluent, accurate, flexible**)

In [ ]:
row = answers_df[(answers_df['question'] == 'What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[2])]
print(row['answer'].values[0])


1. Symptoms/Presentation:  
Depending on the severity and location of the injury, symptoms may include headache, confusion, disorientation, loss of consciousness, memory loss, difficulty with speech and language, weakness or paralysis of specific muscle groups, changes in behavior or personality, and seizures.  

2. Diagnosis Steps:  
Diagnosis is based on a combination of physical examination, medical history, laboratory tests (such as CT or MRI scans), and other diagnostic procedures as appropriate. The severity and location of the injury can be determined using these methods. 

3. Treatment Protocol:  
Treatment for brain injuries depends on the severity and location of the injury, as well as the patient's overall health and medical history. Some common treatments include:  
a) Medications to control symptoms such as headache, seizures, and muscle spasticity.  
b) Rehabilitation therapy (such as physical, occupational, and speech therapy) to help regain lost function and improve co

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

#### Evidence based prompt

**Parameter Combination**
 - temperature: 0.2, top_p: 0.9, top_k: 40, max_tokens: 1024 **(Very factual, less diverse. Useful for guidelines)**


In [ ]:
row = answers_df[(answers_df['question'] == 'What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[0])]
print(row['answer'].values[0])


Precautions:

1. Immobilization: The affected limb must be immobilized using a splint or cast to prevent further damage and promote healing.
2. Pain management: Administer pain medication as prescribed by a physician, taking into account the patient's age, weight, and medical history.
3. Elevation: Elevate the affected limb above heart level to reduce swelling and improve blood flow.
4. Rest: Encourage rest and avoid putting weight on the affected limb to prevent exacerbating the injury.
5. Monitoring: Monitor the patient's vital signs, including pulse, breathing rate, and blood pressure, and seek medical attention if there are any changes or concerns.

Treatment Steps:

1. Stabilization: Stabilize the fracture using a splint or cast to prevent further movement and promote healing.
2. Reduction: If the fracture is displaced, perform reduction to realign the bones and restore normal anatomy.
3. Immobilization: Immobilize the affected limb for 4-6 weeks to allow the fracture to heal.
4.

**Parameter Combination**
 - temperature: 0.5, top_p: 0.8, top_k: 20, max_tokens: 1024 (**Balanced reasoning**)

In [ ]:
row = answers_df[(answers_df['question'] == 'What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[1])]
print(row['answer'].values[0])


For someone who has fractured their leg during a hiking trip, there are several necessary precautions and treatment steps that should be taken to ensure proper care and recovery.

Firstly, it is crucial to immobilize the affected limb to prevent further damage and promote healing. This can be done using a splint or cast, depending on the severity of the fracture. The Merck Manual (Nineteenth Edition) recommends that the splint or cast should be applied in a position that maintains the normal anatomical alignment of the limb.

Secondly, it is essential to manage pain effectively. The patient should be given appropriate analgesics, such as nonsteroidal anti-inflammatory drugs (NSAIDs) or opioids, as recommended by their healthcare provider. It is important to monitor the patient's pain levels and adjust the medication accordingly.

Thirdly, it is crucial to ensure proper wound care. The wound should be cleaned and covered with a sterile dressing, and any signs of infection, such as redn

**Parameter Combination**
 - temperature: 0.7, top_p: 0.95, top_k: 50, max_tokens: 1024 (**Fluent, accurate, flexible**)

In [ ]:
row = answers_df[(answers_df['question'] == 'What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?') & (answers_df['prompt_variant'] == 0) & (answers_df['params'] == param_combos[2])]
print(row['answer'].values[0])


Precautions:

1. Stop all physical activity and rest the affected leg immediately.
2. Apply cold compresses to reduce pain and swelling.
3. Immobilize the injured leg using a splint or brace to prevent further injury.
4. Monitor vital signs, such as pulse, breathing rate, and blood pressure.
5. Keep the affected limb elevated to reduce swelling.
6. Seek medical attention as soon as possible, especially if there are any signs of infection or neurological damage.

Treatment Steps:

1. X-rays should be taken to confirm the fracture and determine the severity of the injury.
2. The affected limb should be immobilized for 6-8 weeks using a cast or brace.
3. Pain management with nonsteroidal anti-inflammatory drugs (NSAIDs) or opioids may be necessary.
4. Physical therapy may be required to restore strength and mobility after the cast or brace is removed.
5. Monitoring for signs of infection, such as redness, swelling, or increased pain, should be done regularly.

Considerations for Care and

#### Structured prompt

**Parameter Combination**
 - temperature: 0.2, top_p: 0.9, top_k: 40, max_tokens: 1024 **(Very factual, less diverse. Useful for guidelines)**

In [ ]:
row = answers_df[(answers_df['question'] == 'What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[0])]
print(row['answer'].values[0])


1. Symptoms/Presentation: A person with a fractured leg may experience severe pain, swelling, bruising, and difficulty moving the affected limb. They may also have an open wound or deformity if the bone is displaced.
2. Diagnosis Steps: The diagnosis of a fractured leg should be confirmed by X-ray imaging. Other injuries such as nerve or blood vessel damage should also be ruled out.
3. Treatment Protocol: Immobilization of the affected limb with a splint, cast, or brace is essential to promote healing and prevent further injury. Pain management with medication is crucial to ensure the patient's comfort and mobility. In some cases, surgical intervention may be necessary to realign and stabilize the bone.
4. Cautions / Follow-up: The patient should be monitored for signs of infection, such as redness, swelling, or increased pain. They should also be advised to avoid putting weight on the affected limb and to follow a rehabilitation program to regain strength and mobility. Regular follow

**Parameter Combination**
 - temperature: 0.5, top_p: 0.8, top_k: 20, max_tokens: 1024 (**Balanced reasoning**)

In [ ]:
row = answers_df[(answers_df['question'] == 'What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[1])]
print(row['answer'].values[0])


1. Symptoms/Presentation: A person with a fractured leg may experience severe pain, swelling, bruising, and difficulty moving the affected limb. They may also have an open wound or deformity at the site of the fracture.
2. Diagnosis Steps: The diagnosis of a fractured leg is typically made through a physical examination and imaging studies such as X-rays or CT scans.
3. Treatment Protocol: Immediate treatment for a fractured leg includes immobilization of the affected limb using a splint or cast, application of ice to reduce swelling, and elevation of the affected limb above heart level. Pain management may involve administering analgesics such as acetaminophen or nonsteroidal anti-inflammatory drugs (NSAIDs). Surgical intervention may be necessary in some cases to realign and stabilize the bones.
4. Cautions / Follow-up: It is important to monitor the patient for signs of infection, such as increased pain, swelling, or redness around the site of the fracture. Regular follow-up appoin

**Parameter Combination**
 - temperature: 0.7, top_p: 0.95, top_k: 50, max_tokens: 1024 (**Fluent, accurate, flexible**)

In [ ]:
row = answers_df[(answers_df['question'] == 'What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?') & (answers_df['prompt_variant'] == 1) & (answers_df['params'] == param_combos[2])]
print(row['answer'].values[0])


1. Symptoms/Presentation:  
A person with a fractured leg may present with signs of pain, swelling, bruising, difficulty moving the affected limb, and deformity. They may also experience numbness or tingling sensations in the affected area.
2. Diagnosis Steps:  
To diagnose a fractured leg, the healthcare provider will typically perform a physical examination and take an X-ray to confirm the presence of a bone fracture. Other imaging tests such as CT or MRI scans may also be ordered to further evaluate the extent of the injury.
3. Treatment Protocol:  
Treatment for a fractured leg will depend on the severity and location of the injury, but may include the following steps:
	* Immobilization of the affected limb using a cast or brace to promote healing and prevent further damage.
	* Managing pain with medication such as nonsteroidal anti-inflammatory drugs (NSAIDs) or opioids.
	* Elevating the affected limb above heart level to reduce swelling.
	* Monitoring for signs of infection, suc

### Observations (Prompt Engineering + LLM Tuning)

**Structure & Format**
- Earlier: narrative & general  
- Now: organized by parameter sets  
- Each combo highlights different response aspects  

**Parameter Combinations**
1. **Conservative**: Highly factual, guideline-focused, rigid structure  
2. **Balanced**: Academic tone, scientific citations, reference sections  
3. **Flexible**: Natural flow, accurate but less rigid, better contextualization  

**Notable Improvements**
- **Citation Quality**: Explicit, numbered references, credible sources  
- **Structural Consistency**: Standardized format, clear sections & hierarchy  
- **Clinical Focus**: More evidence-based, timing clarity, prioritized interventions  

**Areas of Distinction**  
- **Sources**: Previous responses were general, while parameter-based responses explicitly cited references.  
- **Structure**: Previous responses were more fluid, while parameter-based responses were highly organized.  
- **Detail**: Previous responses had consistent detail, while parameter-based responses varied depending on the parameters.  
- **Depth**: Previous responses maintained moderate technical depth, while parameter-based responses varied in depth with different parameter settings.  


**Recommendation**
- Parameter-based tuning + Prompt engineering enables **controlled, evidence-backed, and source-validated** responses.  
- Especially valuable for **medical/clinical contexts** where accuracy and credibility are crucial.

## Data Preparation for RAG

### Loading the Data

In [ ]:
pdf_file = "/content/drive/MyDrive/medical_diagnosis_manual.pdf"

pdf_loader = PyPDFLoader(pdf_file)

In [ ]:
import pickle

cache_file = "/content/drive/MyDrive/mercks_manual_pyloaderobj.pkl"
if os.path.exists(cache_file):
    with open(cache_file, "rb") as f:
        documents = pickle.load(f)
else:
  documents = pdf_loader.load()
  with open(cache_file, "wb") as f:
      pickle.dump(documents, f)

### Data Overview

#### Checking the first 5 pages

In [ ]:
# Inspect first 5 pages
for i, doc in enumerate(documents[:5]):
    print(f"\n\n\n---------- Page {i+1} ----------")
    print(doc.page_content[:900])  # print first 500 characters of page




---------- Page 1 ----------
harshk2010@gmail.com
7OSEGCMQD0
This file is meant for personal use by harshk2010@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.




---------- Page 2 ----------
harshk2010@gmail.com
7OSEGCMQD0
This file is meant for personal use by harshk2010@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.




---------- Page 3 ----------
Table of Contents
1
Front  
  ................................................................................................................................................................................................................
1
Cover  
  .......................................................................................................................................................................................................
2
Front Matter  
  ..............................................................................

#### Checking the number of pages

In [ ]:
print(f"Total pages loaded: {len(documents)}")

Total pages loaded: 4114


### Embedding

In [ ]:
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### Data Chunking

In [ ]:
# Recursive split to fit LLM context
recursive_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,        # Final LLM-friendly chunk size
    chunk_overlap=75       # Maintain context across chunks
)

document_chunks = recursive_splitter.split_documents(documents)

### Vector Database

In [ ]:
report = "medical_assistant_qna"
persist_dir = "/content/drive/MyDrive/chroma_db"

if not os.path.exists(persist_dir):
    vectorstore = Chroma.from_documents(
        document_chunks,
        embedding_model,
        collection_name=report,
        persist_directory=persist_dir
    )
    vectorstore.persist()
else:
    vectorstore = Chroma(
        collection_name=report,
        embedding_function=embedding_model,
        persist_directory=persist_dir
    )


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


### Retriever

In [ ]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 6}
)

### System and User Prompt Template

In [ ]:
qna_system_message = """
You are a highly knowledgeable medical assistant.
Use only the information in ###Context to answer the question.
Answer in clear, concise language suitable for medical professionals or students.
- Do not make assumptions beyond the context.
- If the answer cannot be determined from the context, respond "I don't know".
- Include relevant treatment names, doses, or procedures only if present in the context.
- For multi-part questions, answer each part clearly and separately.
"""

In [ ]:
qna_user_message_template = """
Consider the following ###Context and ###Question
###Context
{context}

###Question
{question}

Answer:

"""

### Response Function

In [ ]:
def generate_rag_response(user_input,k=3,max_tokens=512,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = lcpp_llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

## Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
print(generate_rag_response("What is the protocol for managing sepsis in a critical care unit?"))

Llama.generate: prefix-match hit


The management of sepsis in a critical care unit should be guided by the Surviving Sepsis Campaign (SSC) guidelines. The SSC recommends the following protocol for managing sepsis:

1. Early recognition and diagnosis of sepsis:
	* Use clinical criteria such as SIRS or QSEOR to identify patients with suspected infection.
	* Perform blood cultures and obtain other diagnostic tests (e.g., complete blood count, serum lactate) to confirm the diagnosis of sepsis.
2. Early administration of antibiotics:
	* Use broad-spectrum antibiotics effective against common causes of sepsis, such as gram-negative bacteria and Staphylococcus aureus.
	* Administer antibiotics within the first hour of recognition of sepsis.
3. Fluid resuscitation:
	* Use crystalloid fluids (e.g., 0.9% saline) to restore intravascular volume and improve perfusion.
	* Administer fluid resuscitation promptly, ideally within the first hour of recognition of sepsis.
4. Monitoring and management of vital signs:
	* Monitor blood pre

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
print(generate_rag_response("What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"))

Llama.generate: prefix-match hit


The common symptoms of appendicitis include severe pain in the abdomen, nausea, vomiting, fever, loss of appetite, and constipation. However, these symptoms are not always present or may be mild in some cases. Appendicitis is a medical emergency that requires prompt surgical intervention.

Medications can help manage the symptoms but cannot cure appendicitis. Antibiotics may be given to prevent infection, and pain medication may be prescribed to alleviate discomfort. However, these medications do not address the underlying cause of the inflammation.

The surgical procedure for treating appendicitis is an appendectomy, which involves removing the inflamed appendix. The two types of appendectomies are open and laparoscopic.

Open Appendectomy: This is the traditional method, where a single incision is made in the lower right abdomen to remove the inflamed appendix.

Laparoscopic Appendectomy: This is a minimally invasive procedure, where three small incisions are made in the abdomen, and

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
print(generate_rag_response("What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"))

Llama.generate: prefix-match hit


Based on the provided context, sudden patchy hair loss, commonly seen as localized bald spots on the scalp, can be caused by various factors such as alopecia areata, lichen planopilaris, or scarring alopecia. The most effective treatments for addressing this condition depend on the underlying cause.

For alopecia areata, topical corticosteroids, intralesional corticosteroid injections, or oral immunosuppressants may be prescribed to suppress the immune system and promote hair growth. In cases of lichen planopilaris, treatment with oral antimalarials, topical corticosteroids, or immunomodulators may be effective.

For scarring alopecia, long-acting oral tetracycline in combination with a potent topical corticosteroid may be recommended. Additionally, addressing any underlying conditions such as atopy, psoriasis, or systemic lupus erythematosus can help resolve the hair loss.

It is essential to consult a dermatologist for an accurate diagnosis and appropriate treatment plan. The dermato

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
print(generate_rag_response("What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"))

Llama.generate: prefix-match hit


Based on the information provided in the context, the following treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function:

1. Rehabilitation services should be planned early, including physical, occupational, and speech therapy, skill-building activities, and counseling to meet the patient's social and emotional needs.
2. Surgery may be necessary in patients with more severe injury to place monitors to track and treat intracranial pressure, decompress the brain if intracranial hematomas are present, or remove intracranial hematomas.
3. Cognitive and emotional support is crucial for patients with traumatic brain injury (TBI), including family education and early intervention by rehabilitation specialists to maximize functional recovery.
4. Specific syndromes such as agnosia, amnesia, aphasia, and apraxia may require individualized treatment approaches, including neuropsychologic testing 

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
print(generate_rag_response("What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"))

Llama.generate: prefix-match hit


Based on the provided context, here is the answer to the question:

For a person who has fractured their leg during a hiking trip, necessary precautions and treatment steps include:

1. Immobilization: The affected limb should be immobilized using a splint or cast to prevent further injury and promote healing.
2. Pain management: Analgesics and NSAIDs should be prescribed to manage pain and reduce inflammation.
3. Elevation: The affected limb should be elevated above the heart level to reduce swelling and promote blood flow.
4. Cleanliness: The wound area should be kept clean and dry to prevent infection.
5. Monitoring: Regular monitoring of the patient's condition, including pain levels, swelling, and wound healing, is crucial to ensure proper recovery.
6. Follow-up care: Follow-up appointments with a healthcare professional should be scheduled to assess progress and address any complications or concerns.

Considerations for care and recovery include:

1. Rest: The patient should rest

### Observations for RAG Based LLM System

**Overall Structure**
- Responses are concise and focused  
- Direct medical information without heavy subdivisions  
- Limited flexibility compared to parameter-based responses  

**Key Characteristics**
1. **Information Density**  
   - High concentration of relevant content  
   - Minimal repetition  
   - Streamlined presentation  

2. **Source Integration**  
   - Fewer explicit citations compared to parameter-based responses  

3. **Response Format**  
   - Shorter, bullet-point style answers  
   - Less hierarchical structuring  

**Strengths**
- **Clarity**: Direct answers, clear actionable steps, focused guidance  
- **Practicality**: Immediate applicability, less jargon, quick reference style  
- **Consistency**: Uniform structure, steady depth, reliable formatting  

**Limitations**
- **Depth**: Less detailed, fewer alternative approaches, minimal scientific context  
- **Customization**: Fixed format, no parameter-driven variation, less adaptable to complexity  

**Comparison to Earlier Results**
- **Length**: RAG responses are shorter, earlier ones were more extensive  
- **Structure**: RAG uses bullet points, earlier systems used hierarchical sections  
- **Citations**: RAG has minimal sourcing, earlier responses emphasized explicit references  
- **Detail & Depth**: RAG favors practicality and clarity, parameter-based responses provided more depth and variation  

**Recommendation**
- RAG is ideal for **quick medical reference**, offering clarity and efficiency.  
- For complex cases, it should be **supplemented with parameter-based or structured responses** to add depth, context, and citations.  


### Fine-tuning

- From the earlier evaluation on all queries, first question seemed to have a truncated answer whereas all others seem complete.
- **"What is the protocol for managing sepsis in a critical care unit?"** - this is good candidate for fine tuning evaluation.

In [ ]:
questions = [
    "What is the protocol for managing sepsis in a critical care unit?"
    #"What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    #"What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    #"What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    #"What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
]

param_combos_rag = [
    {"temperature": 0.2, "top_p": 0.9, "top_k": 40, "max_tokens": 1024, "k": 6}, # Combo 1: Very factual, low creativity. Good for strict protocols and guidelines.
    {"temperature": 0.3, "top_p": 0.95, "top_k": 50, "max_tokens": 512, "k": 8}, # Combo 2: Slightly more flexible, can include reasoning. Useful for symptoms or multi-step procedures.
    {"temperature": 0.5, "top_p": 0.85, "top_k": 30, "max_tokens": 512, "k": 10}, # Combo 3: Balanced factual recall and reasoning. Good for multi-part questions.
    {"temperature": 0.7, "top_p": 0.95, "top_k": 60, "max_tokens": 1024, "k": 10}, # Combo 4: More fluent and flexible, generates longer answers with inferred reasoning.
    {"temperature": 0.8, "top_p": 0.9, "top_k": 50, "max_tokens": 1024, "k": 12}, # Combo 5: Highly creative/exploratory within retrieved context. Good for comprehensive coverage.
]

results = []

for qi, q in enumerate(questions):
    for ci, params in enumerate(param_combos_rag):
        answer = generate_rag_response(
            user_input=q,
            k=params["k"],
            max_tokens=params["max_tokens"],
            temperature=params["temperature"],
            top_p=params["top_p"],
            top_k=params["top_k"]
        )
        results.append({
            "question": q,
            "params": params,
            "answer": answer
        })

import pandas as pd
answers = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


In [ ]:
file_path = '/content/drive/MyDrive/ragtuning_results.pkl'
answers.to_pickle(file_path)

In [ ]:
answers.head(5)

,question,params,answer
0,What is the protocol for managing sepsis in a critical care unit?,"{'temperature': 0.2, 'top_p': 0.9, 'top_k': 40, 'max_tokens': 1024, 'k': 6}","The protocol for managing sepsis in a critical care unit involves several key components, including:\n\n1. Early recognition and prompt treatment of sepsis, including administration of broad-spectrum antibiotics and fluid resuscitation to raise mean arterial pressure (MAP) to at least 65 mmHg.\n2. Aggressive fluid resuscitation with 0.9% saline until central venous pressure (CVP) reaches 8 mmHg or pulmonary artery pressure (PAOP) reaches 12-15 mmHg, as well as the use of vasopressors such as dopamine or norepinephrine if necessary.\n3. Monitoring of vital signs, including temperature, blood pressure, heart rate, and respiratory rate, as well as regular assessment of organ function and fluid balance.\n4. Management of hypotension with vasopressors, and management of hypoxia with oxygen therapy and mechanical ventilation if necessary.\n5. Protocols for the administration of antibiotics, including the use of broad-spectrum antibiotics and the addition of a third-generation cephalosporin or carbapenem if Pseudomonas aeruginosa is suspected.\n6. Protocols for the management of sepsis-induced organ dysfunction, including the use of renal replacement therapy (RRT) and mechanical ventilation as needed.\n7. Close monitoring of blood glucose levels and the administration of insulin if necessary to maintain euglycemia.\n8. Protocols for the management of sepsis in special populations, such as neonates, children, and pregnant women.\n9. Regular reassessment of patients and adjustment of therapy as needed based on clinical response and laboratory results.\n10. Early recognition and management of complications such as acute respiratory distress syndrome (ARDS), acute kidney injury (AKI), and shock liver.\n\nIt is important to note that the specifics of the protocol may vary depending on the individual patient and the institution's policies and procedures."
1,What is the protocol for managing sepsis in a critical care unit?,"{'temperature': 0.3, 'top_p': 0.95, 'top_k': 50, 'max_tokens': 512, 'k': 8}","The management of sepsis in a critical care unit should be guided by the Surviving Sepsis Campaign (SSC) guidelines. The key components of the SSC guidelines are:\n\n1. Early recognition and diagnosis of sepsis, including the use of clinical criteria such as SIRS or QSOFA.\n2. Rapid administration of antibiotics within the first hour of recognition of sepsis.\n3. Aggressive fluid resuscitation with crystalloids or vasopressors to achieve a mean arterial pressure (MAP) of 65 mmHg or higher.\n4. Use of vasopressors to maintain MAP if fluid resuscitation is not sufficient.\n5. Early recognition and management of organ dysfunction, including acute respiratory distress syndrome (ARDS), acute kidney injury (AKI), and cardiovascular instability.\n6. Use of steroids in patients with sepsis-induced hypotension who have a clinical presentation suggestive of severe sepsis or septic shock.\n7. Protocolized management of fever and hyperthermia, including the use of antipyretics and cooling measures as needed.\n8. Close monitoring of blood glucose levels and management of hyperglycemia or hypoglycemia as needed.\n9. Regular assessment of fluid balance and adjustment of fluid therapy accordingly.\n10. Early recognition and management of complications such as pneumonia, urinary tract infections, and bloodstream infections.\n\nIt is important to note that the management of sepsis is highly individualized and may vary depending on the patient's specific needs and clinical presentation. The SSC guidelines should be used as a general framework for managing sepsis, but the care team should be prepared to adjust the management plan as needed based on the patient's response to treatment and any changes in their clinical status."
2,What is the protocol for managing sepsis in a critical care unit?,"{'

#### Query: What is the protocol for managing sepsis in a critical care unit?

**Parameter Combination**
 - temperature: 0.2, top_p: 0.9, top_k: 40, max_tokens: 1024, k: 6 **(Very factual, low creativity. Good for strict protocols and guidelines.)**


In [ ]:
print(answers.iloc[0]['answer'])

The protocol for managing sepsis in a critical care unit involves several key components, including:

1. Early recognition and prompt treatment of sepsis, including administration of broad-spectrum antibiotics and fluid resuscitation to raise mean arterial pressure (MAP) to at least 65 mmHg.
2. Aggressive fluid resuscitation with 0.9% saline until central venous pressure (CVP) reaches 8 mmHg or pulmonary artery pressure (PAOP) reaches 12-15 mmHg, as well as the use of vasopressors such as dopamine or norepinephrine if necessary.
3. Monitoring of vital signs, including temperature, blood pressure, heart rate, and respiratory rate, as well as regular assessment of organ function and fluid balance.
4. Management of hypotension with vasopressors, and management of hypoxia with oxygen therapy and mechanical ventilation if necessary.
5. Protocols for the administration of antibiotics, including the use of broad-spectrum antibiotics and the addition of a third-generation cephalosporin or carb

**Parameter Combination**
 - temperature: 0.3, top_p: 0.95, top_k: 50, max_tokens: 512, k: 8 **(Slightly more flexible, can include reasoning. Useful for symptoms or multi-step procedures.)**


In [ ]:
print(answers.iloc[1]['answer'])

The management of sepsis in a critical care unit should be guided by the Surviving Sepsis Campaign (SSC) guidelines. The key components of the SSC guidelines are:

1. Early recognition and diagnosis of sepsis, including the use of clinical criteria such as SIRS or QSOFA.
2. Rapid administration of antibiotics within the first hour of recognition of sepsis.
3. Aggressive fluid resuscitation with crystalloids or vasopressors to achieve a mean arterial pressure (MAP) of 65 mmHg or higher.
4. Use of vasopressors to maintain MAP if fluid resuscitation is not sufficient.
5. Early recognition and management of organ dysfunction, including acute respiratory distress syndrome (ARDS), acute kidney injury (AKI), and cardiovascular instability.
6. Use of steroids in patients with sepsis-induced hypotension who have a clinical presentation suggestive of severe sepsis or septic shock.
7. Protocolized management of fever and hyperthermia, including the use of antipyretics and cooling measures as need

**Parameter Combination**
 - temperature: 0.5, top_p: 0.85, top_k: 30, max_tokens: 512, k: 10 **(Balanced factual recall and reasoning. Good for multi-part questions.)**


In [ ]:
print(answers.iloc[2]['answer'])

The protocol for managing sepsis in a critical care unit should be guided by the specific needs of each patient and may involve a combination of medications, fluids, and other interventions. However, some general principles can be applied to all patients with sepsis. These include:

1. Early recognition and aggressive management of sepsis: Sepsis should be recognized early and treated promptly to prevent progression to severe sepsis or septic shock.
2. Use of broad-spectrum antibiotics: Broad-spectrum antibiotics should be used to cover the most common pathogens causing sepsis, such as gram-negative bacteria and methicillin-resistant Staphylococcus aureus (MRSA).
3. Fluid resuscitation: Patients with sepsis may require aggressive fluid resuscitation to maintain mean arterial pressure (MAP) of at least 65 mmHg and urine output of at least 0.5 mL/kg/h.
4. Vasopressor support: If fluid resuscitation is not sufficient to maintain MAP, vasopressor support may be needed to maintain mean arte

**Parameter Combination**
 - temperature: 0.7, top_p: 0.95, top_k: 60, max_tokens: 1024, k: 10 **(More fluent and flexible, generates longer answers with inferred reasoning.)**


In [ ]:
print(answers.iloc[3]['answer'])

The protocol for managing sepsis in a critical care unit includes:

1. Early recognition of sepsis based on clinical signs and symptoms, such as fever, tachycardia, tachypnea, and confusion.
2. Performance of blood cultures and serial measurements of white blood cell count, serum lactate, and hemoglobin levels to confirm the diagnosis of sepsis and monitor progression of the disease.
3. Administration of broad-spectrum antibiotics, targeting the most likely pathogens based on the patient's medical history, physical examination, and laboratory results.
4. Aggressive fluid resuscitation with 0.9% saline, aiming to raise mean arterial pressure (MAP) to at least 65 mmHg and central venous oxygen saturation (ScvO2) to at least 70%.
5. Close monitoring of vital signs, including MAP, pulmonary artery pressure (PAP), and central venous pressure (CVP), as well as serial measurements of serum lactate and hemoglobin levels.
6. Use of vasopressors, such as dopamine or norepinephrine, to maintain m

**Parameter Combination**
 - temperature: 0.8, top_p: 0.9, top_k: 50, max_tokens: 1024, k: 12 **(Highly creative/exploratory within retrieved context. Good for comprehensive coverage.)**


In [ ]:
print(answers.iloc[4]['answer'])

According to the information provided in the context, the following is the protocol for managing sepsis in a critical care unit:

1. Broad-spectrum antibiotics (modified by culture results)
2. Drainage of abscesses and excision of necrotic tissue
3. Normalization of blood glucose levels
4. Replacement-dose corticosteroids
5. Frequent monitoring of systemic pressure, CVP, PAOP, pulse oximetry, ABGs, blood glucose, lactate, and electrolyte levels
6. Renal function assessment
7. Fluid resuscitation with 0.9% saline until CVP reaches 8 mm Hg (10 cm H2O) or PAOP reaches 12 to 15 mm Hg
8. Oliguria with hypotension is not a contraindication to vigorous fluid resuscitation
9. The quantity of fluid required may far exceed the normal blood volume and reach 10 L over 4 to 12 h
10. PAOP or echocardiography can identify limitations in left ventricular function and incipient pulmonary edema due to fluid overload
11. If a patient with septic shock remains hypotensive after CVP or PAOP has been raised

#### Observations

- First combination which was aimed at being Very factual, low creativity. Good for strict protocols and guidelines.

- temperature: 0.2, top_p: 0.9, top_k: 40, max_tokens: 1024, k: 6

- We will do the output evaluation using these parameters.

## Output Evaluation

In [ ]:
groundedness_rater_system_message = """

You will be presented a ###Question, ###Context used by the AI system and AI generated ###Answer.

Your task is to judge the extent to which the ###Answer is derived from ###Context.

Rate it 1 - if The ###Answer is not derived from the ###Context at all
Rate it 2 - if The ###Answer is derived from the ###Context only to a limited extent
Rate it 3 - if The ###Answer is derived from ###Context to a good extent
Rate it 4 - if The ###Answer is derived from ###Context mostly
Rate it 5 - if The ###Answer is is derived from ###Context completely

Please note: Make sure you give a single overall rating in the range of 1 to 5 along with an overall explanation.

"""

In [ ]:
relevance_rater_system_message = """

You will be presented with a ###Question, the ###Context used by the AI system to generate a response, and the AI-generated ###Answer.

Your task is to judge the extent to which the ###Answer is relevant to the ###Question, considering whether it directly addresses the key aspects of the ###Question based on the provided ###Context.

Rate the relevance as follows:
- Rate 1 – The ###Answer is not relevant to the ###Question at all.
- Rate 2 – The ###Answer is only slightly relevant to the **###Question**, missing key aspects.
- Rate 3 – The ###Answer is moderately relevant, addressing some parts of the **###Question** but leaving out important details.
- Rate 4 – The ###Answer is mostly relevant, covering key aspects but with minor gaps.
- Rate 5 – The ###Answer is fully relevant, directly answering all important aspects of the **###Question** with appropriate details from the **###Context**.

Note: Provide a single overall rating in the range of 1 to 5, along with a brief explanation of why you assigned that score.
"""

In [ ]:
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [ ]:
def generate_ground_relevance_response(user_input,k=3,max_tokens=512,temperature=0.2,top_p=0.90,top_k=40):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    response = lcpp_llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = lcpp_llm(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    response_2 = lcpp_llm(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

In [ ]:
questions = [
    "What is the protocol for managing sepsis in a critical care unit?",
    "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
]
for q in questions:
  print(f"Question: {q}\n")
  groundedness_report, relevance_report = generate_ground_relevance_response(q)
  print(groundedness_report, '\n\n', relevance_report)

Question: What is the protocol for managing sepsis in a critical care unit?



Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Based on the provided context, I would rate the answer as follows:

* Not derived from the context (1): The answer does not mention any specific details or information from the provided context.
* Limited extent (2): The answer provides some general information about the management of sepsis, but it does not fully address the question or provide specific guidance based on the context.

The answer could be improved by incorporating more specific details and information from the provided context, such as the use of broad-spectrum antibiotics, fluid resuscitation, and source control. Additionally, the answer could benefit from a more detailed explanation of how these interventions should be tailored to the individual patient's needs and clinical presentation. 

   Based on the provided context, I would rate the answer as follows:

Rating: 4 (Moderately relevant)

The answer provides a detailed list of the steps involved in managing sepsis in a critical care unit, including early recogni

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Based on the given text, I would rate the answer as follows:

Question 1: What are the common symptoms for appendicitis?
Rating: 5 (Completely accurate)

Question 2: Can appendicitis be cured via medicine?
Rating: 1 (Not at all accurate)

Question 3: What surgical procedure should be followed to treat appendicitis?
Rating: 5 (Completely accurate)

Overall, the answer provides a comprehensive and accurate response to all three questions based on the provided context. 

   Rating: 4 (Fully Relevant)

Your answer fully addresses all aspects of the question, providing clear and concise information about the common symptoms of appendicitis, the ineffectiveness of medication in treating the condition, and the recommended surgical procedure for treatment. Your explanation is well-organized and easy to follow, making use of key terms and concepts from the provided context. Overall, your answer demonstrates a strong understanding of the topic and effectively addresses the question.
Question: 

Llama.generate: prefix-match hit


ValueError: Requested tokens (4120) exceed context window of 4096

In [ ]:
questions = [
    "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
]
for q in questions:
  print(f"Question: {q}\n")
  groundedness_report, relevance_report = generate_ground_relevance_response(q)
  print(groundedness_report, '\n\n', relevance_report)

Question: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?



Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Based on the provided context, I would rate the answer as follows:

The answer provides a comprehensive list of possible causes of sudden patchy hair loss, including both medical and non-medical conditions. The answer also mentions various treatment options for each condition, which is relevant to the question. However, the answer does not provide a specific rating or ranking of the effectiveness of each treatment option, which could have been helpful in providing a more detailed assessment.

Over 

   Based on the provided context, I would rate the answer as
Question: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?



Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Based on the provided context, I would rate the answer as follows:

1. Rehabilitation services: 5 (completely derived from the context)
2. Surgery: 4 (mostly derived from the context)
3. Early intervention by rehabilitation specialists: 5 (completely derived from the context)
4. Specific rehabilitation therapy: 4 (mostly derived from the context)
5. Cognitive therapy: 4 (mostly derived from the context)
6. Immobilization of the affected area: 5 (completely derived from the context)
7. Prevention of complications: 4 (mostly derived from the context)
8. Prognosis for recovery: 4 (mostly derived from the context)
9. Treatment by qualified speech therapists: 5 (completely derived from the context)
10. Use of augmentative communication devices: 4 (mostly derived from the context)

Overall, the answer is well-supported by the provided context and accurately reflects the recommended treatments for patients with physical injury to brain tissue resulting in temporary or permanent impairment o

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Based on the information provided in the context, I would rate the answer as 4 (is derived from the context mostly). The answer provides a comprehensive list of necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, including immobilization, pain management, elevation, cleanliness, monitoring, and follow-up care. Additionally, the answer considers factors such as rest, ice, compression, elevation, physical therapy, and follow-up appointments for the patient's care and recovery. While the answer is not a verbatim repetition of the context, it is clear that the information provided in the context was used to formulate the answer. Therefore, I would rate the answer as 4 (is derived from the context mostly). 

   Based on the information provided in the context, I would rate the answer as follows:

Rating: 4 (Moderately relevant)

The answer provides some relevant information about the necessary precautions and treatment steps for a perso

## Observation Summary: Groundedness and Relevance Ratings

### 1. **Sepsis Management (Critical Care Unit)**
- **Groundedness:** Limited : The answer did not fully leverage provided context (missing specifics like broad-spectrum antibiotics, fluid resuscitation, source control).  
- **Relevance:** Moderate (Rating: 4) : Answer listed general steps but lacked patient-specific tailoring.  
- **Observation:** Answers tended to be **generic** and not tightly tied to context. Improvement would come from explicitly grounding in the reference manual.


### 2. **Appendicitis (Symptoms, Cure via Medicine, Surgical Procedure)**
- **Groundedness:** Mixed : Strong on symptoms and surgical treatment (Rating: 5 each), but poor on medicine-related aspect (Rating: 1).  
- **Relevance:** High (Rating: 4) : Answer fully addressed the multi-part question in structure and clarity.  
- **Observation:** When multiple sub-questions are asked, the model provides **well-structured, organized responses**, though occasional inaccuracies appear on nuanced points (e.g., medical vs. surgical options).  


### 3. **Sudden Patchy Hair Loss (Causes & Treatments)**
- **Groundedness:** Reasonable : Covered both medical and non-medical causes and treatments.  
- **Relevance:** Moderate : Comprehensive, but lacked prioritization or ranking of treatment effectiveness.  
- **Observation:** Responses were **complete but shallow**, giving breadth over depth. More value could be added by distinguishing **first-line vs. secondary interventions**.  

### 4. **Brain Injury (Treatment for Impairment of Function)**
- **Groundedness:** Strong : Most recommended treatments were directly or mostly derived from the Merck Manual context.  
  - Rehabilitation services: **5**  
  - Surgery: **4**  
  - Early intervention by rehabilitation specialists: **5**  
  - Specific rehabilitation therapy: **4**  
  - Cognitive therapy: **4**  
  - Immobilization of the affected area: **5**  
  - Prevention of complications: **4**  
  - Prognosis for recovery: **4**  
  - Speech therapy: **5**  
  - Use of augmentative communication devices: **4**  
- **Relevance:** High : The answer was well-aligned with the question and structured around appropriate categories of intervention.  
- **Observation:** This example shows the model can generate **highly grounded and context-aligned responses** when information is available and used correctly.  

## 5. **Leg Fracture During Hiking (Precautions and Treatment Steps)**
- **Groundedness:** Moderate : The generated response drew from the context but missed some critical details.  
  - Immobilization: **4**  
  - Pain management: **4**  
  - Elevation and compression: **4**  
  - Cleanliness and infection prevention: **4**  
  - Monitoring and follow-up care: **4**  
  - Physical therapy and recovery considerations: **4**  
- **Relevance:** Medium to High : The response covered important aspects (immobilization, RICE protocol, follow-up), but lacked specificity on fracture severity, surgical interventions, and infection risks.  
- **Observation:** While the answer captured the **general management approach** for fractures, it did not fully address all aspects of the question. More precise details about **types of fractures, surgical needs, and long-term recovery** would improve both groundedness and relevance.  

### Overall Insights
- **Strengths:**
  - Answers are usually **structured and relevant**.  
  - Model handles **multi-part queries well**.  
  - Coverage is broad across medical domains.  

- **Weaknesses:**
  - Groundedness suffers when context is not explicitly used.  
  - Answers can be **generic**, lacking clinical nuance.  
  - Token and parameter constraints visibly degrade output quality.

## Actionable Insights and Business Recommendations

**Observations & Insights**
- The RAG-based assistant delivers **clear, concise, and practical** medical guidance drawn from the Merck Manual.  
- Responses are **streamlined** with high information density, making them suitable for **quick reference use cases** (e.g., frontline healthcare workers, patients needing rapid guidance).  
- Compared to parameter-based or structured responses, the RAG system **sacrifices depth, citations, and adaptability** in favor of **clarity and speed**.  
- Consistency across responses is high, but the lack of customization may limit usefulness in **complex or nuanced medical cases**.  

**Recommendations**
1. **Dual-Mode Strategy**  
   - Use RAG-based responses as the **default quick-reference layer**.  
   - Provide an option to switch to **detailed, parameter-based explanations** when complexity or citation requirements are higher.  

2. **Enhance Source Integration**  
   - Incorporate **inline citations from the Merck Manual** to increase trust and scientific credibility.  

3. **Contextual Adaptation**  
   - Enable the system to adjust depth based on **user type** (e.g., clinician vs. patient) or **case complexity**.  

4. **Business Value**  
   - Position the RAG system as a **lightweight, efficient medical assistant** for fast guidance.  
   - Differentiate by offering **scalable depth**: quick answers for most users, detailed protocols for professionals.  
   - This layered approach can broaden adoption across **consumer health apps, clinical decision support tools, and training platforms**.  


<font size=6 color='blue'>Power Ahead</font>
___